In [1]:
!sudo apt-get update -y
!sudo apt-get install python3.8
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!apt-get install python3.8-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python3 get-pip.py

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,966 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:13 http://

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install openai-whisper torch openpyxl pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 20.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 139.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 142.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 15

In [4]:
import os
import whisper
import torch
import gc

In [ ]:

# Set up parameters
xlsx_file = "JM.xlsx"
audio_folder = "drive/MyDrive/Joyful_Melodies"

# Define the serial numbers
western_serials = [691, 615, 601, 581,424, 325, 312, 194 ]
indian_serials = [678, 657, 547, 406, 362, 353 ]

def transcribe_audio_whisper(audio_file: str, model_name: str = "medium") -> list:
    """Transcribe the given audio file using the Whisper model locally."""
    print(f"Loading Whisper model: {model_name}")
    model = whisper.load_model(model_name)

    print(f"Transcribing audio file: {audio_file}")
    with torch.no_grad():
        audio = whisper.load_audio(audio_file)
        result = model.transcribe(audio, language='en', word_timestamps=True)

    transcription = []
    for segment in result['segments']:
        for word in segment['words']:
            transcription.append((word['word'], (word['start'], word['end'])))

    del result
    gc.collect()
    return transcription

def process_serials(serials, prefix):
    for serial_number in serials:
        audio_files = [f for f in os.listdir(audio_folder) if f.startswith(f"{serial_number:03d}")]
        if not audio_files:
            print(f"No audio file found for serial number {serial_number}")
            continue

        audio_file = os.path.join(audio_folder, audio_files[0])
        print(f"Processing audio file: {audio_file}")

        transcription = transcribe_audio_whisper(audio_file, model_name="large")

        # Save the transcription to a text file
        output_file = f"{prefix}_{serial_number:03d}_transcription.txt"
        with open(output_file, "w") as f:
            for word, (start, end) in transcription:
                f.write(f"{word} ({start:.2f} - {end:.2f})\n")

        print(f"Transcription saved to {output_file}")

# Process Western serials
print("Processing Western serials...")
process_serials(western_serials, "Western")

# Process Indian serials
print("Processing Indian serials...")
process_serials(indian_serials, "Indian")

print("All transcriptions completed.")

Processing Western serials...
Processing audio file: drive/MyDrive/Joyful_Melodies/691 You are My Strength When I am Weak.mp3
Loading Whisper model: large


100%|█████████████████████████████████████| 2.88G/2.88G [05:30<00:00, 9.35MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcribing audio file: drive/MyDrive/Joyful_Melodies/691 You are My Strength When I am Weak.mp3
Transcription saved to Western_691_transcription.txt
Processing audio file: drive/MyDrive/Joyful_Melodies/615 Trust In the Lord.mp3
Loading Whisper model: large


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcribing audio file: drive/MyDrive/Joyful_Melodies/615 Trust In the Lord.mp3
Transcription saved to Western_615_transcription.txt
Processing audio file: drive/MyDrive/Joyful_Melodies/601 Though the Mountains May Fall.mp3
Loading Whisper model: large


...

Transcribing audio file: drive/MyDrive/Joyful_Melodies/406 Make me a servant - wjl D104.mp3
Transcription saved to Indian_406_transcription.txt
Processing audio file: drive/MyDrive/Joyful_Melodies/362 Let me live in yours house  O Lord - wjl A103.mp3
Loading Whisper model: large


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcribing audio file: drive/MyDrive/Joyful_Melodies/362 Let me live in yours house  O Lord - wjl A103.mp3
Transcription saved to Indian_362_transcription.txt
Processing audio file: drive/MyDrive/Joyful_Melodies/353 Just like a child - wjl A74.mp3
Loading Whisper model: large


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcribing audio file: drive/MyDrive/Joyful_Melodies/353 Just like a child - wjl A74.mp3
Transcription saved to Indian_353_transcription.txt
All transcriptions completed.


In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
drive.flush_and_unmount()